## Download Data

In [ ]:
# !wget --no-check-certificate \
#   https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
#   -O cats_and_dogs_filtered.zip
# ! unzip cats_and_dogs_filtered.zip

## System setting and import packages

In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

os.environ["RUNFILES_DIR"] = "/Users/bunnykitty/opt/anaconda3/envs/tf/share/plaidml"
# plaidml might exist in different location. Look for "/usr/local/share/plaidml" and replace in above path

os.environ["PLAIDML_NATIVE_PATH"] = "/Users/bunnykitty/opt/anaconda3/envs/tf/lib/libplaidml.dylib"
# libplaidml.dylib might exist in different location. Look for "/usr/local/lib/libplaidml.dylib" and replace in above path

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

import matplotlib.pyplot as plt
import numpy as np

Using plaidml.keras.backend backend.


## Vectorize the data

In [ ]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="cats_and_dogs_filtered/train",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="cats_and_dogs_filtered/validation", target_size=(224,224))

## Setup NN model

In [ ]:
model_loading_mode = 0

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))
model_loading_mode = 1

In [ ]:
if model_loading_mode == 0 :
    #load entire model
    model = load_model("saved_model/vgg16.hdf5")
else:
    #only load weights
    model.load_weights("saved_model/vgg16_imagenet_weights.h5")

In [ ]:

# from keras.optimizers import Adam
# opt = Adam(lr=0.001)
# model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
# model.summary()
from keras import optimizers
opt=optimizers.SGD(lr=0.0001, momentum=0.9)
model.compile(optimizer=opt, loss = "categorical_crossentropy", metrics=['accuracy'])
model.summary()


## Visualize the model

In [ ]:
# from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model
# model = VGG16()
plot_model(model, to_file='vgg16.png')

## ModelCheckpoint and EarlyStopping

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
# filepath="vgg16_{epoch:02d}-{val_acc:.2f}.hdf5"
filepath="saved_model/vgg16.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             period=1)
early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=20, 
                      verbose=1, 
                      mode='auto')
hist = model.fit_generator(steps_per_epoch=2,
                           generator=traindata, 
                           validation_data= testdata, 
                           validation_steps=2,
                           epochs=100,
                           callbacks=[checkpoint,early])

## visualize

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

## Validate the trained model

In [ ]:
# from keras.preprocessing import image
# img = image.load_img("image3.jpeg",target_size=(224,224))
# img = np.asarray(img)
# import matplotlib.pyplot as plt
# plt.imshow(img)

In [ ]:
from keras.preprocessing import image
testimgpath="cats_and_dogs_filtered/validation/cats/cat.2001.jpg"
testimgpath="testimg/cen.JPG"
img = image.load_img(testimgpath,target_size=(224,224))
img = np.asarray(img)
import matplotlib.pyplot as plt
plt.imshow(img)
img = np.expand_dims(img, axis=0)
filepath="saved_model/vgg16.hdf5"
from keras.models import load_model
saved_model = load_model(filepath)
output = saved_model.predict(img)

if output[0][0] > output[0][1]:
    print("cat")
else:
    print('dog')